* The main idea of this notebook is using retrieval two times.
  * The first time: Get the top-K1 relavent misconceptions to LLM as a reference(using ConstructName + SubjectName).
  * The second time: Get the top-K2(K2 < K1) relavent misconceptions(using ConstructName + SubjectName + Question + Answer + LLM's output).
  * Inference time: ~2 hours
 

Thanks to these great works:
- [Zero-shot w/ LLM feature (LB: 0.180)](https://www.kaggle.com/code/ubamba98/eedi-zero-shot-w-llm-feature-lb-0-180)
- [Infer BGE Synthetic Data](https://www.kaggle.com/code/minhnguyendichnhat/infer-bge-synthetic-data)
- [Fine-tuning bge Train](https://www.kaggle.com/code/sinchir0/fine-tuning-bge-train)

In [1]:
%%time
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl
!pip3 install --no-deps --no-index --force-reinstall /kaggle/input/hf-libraries/tokenizers/tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip3 install --no-deps --no-index --force-reinstall /kaggle/input/hf-libraries/transformers/transformers-4.45.2-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Processing /kaggle/input/hf-libraries/tokenizers/tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
Processing /kaggle/input/hf-libraries/transformers/transformers-4.45.2-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
CPU times: user 2.9 s, sys: 666 ms, total: 3.56 s
Wall time: 4min 14s


In [1]:
import os, math, numpy as np
import os
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import re, gc
import torch
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# pd.set_option('display.max_rows', 300)

/Users/lifeng/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Metric

In [3]:
%%writefile eedi_metrics.py

# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

Writing eedi_metrics.py


## Prepare dataframe

In [4]:
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

model_path = "/kaggle/input/qwen/transformers/qwen2.5-32b-instruct-awq-mmim-ref/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)

df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1).sample(100, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")

# first retrieval

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

if not IS_SUBMISSION:
    df_ret = df_train.copy()
else:
    df_ret = df_test.copy()
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')
df_ret.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,1700,321,Divide two decimals with the same number of de...,224,Multiplying and Dividing with Decimals,D,\( 0.9 \div 0.3= \),\( 0.3 \),\( 9.3 \),\( 0.333 \ldots \),\( 3 \),153.0,-1.0,2359.0,-1.0
1,1488,469,Understand the notation for powers,245,"Squares, Cubes, etc",C,To calculate \( 53^{2} \) you need to do...,\( 53+2 \),\( 53 \times 2 \),\( 53 \times 53 \),\( 532 \times 1 \),2445.0,2316.0,-1.0,-1.0
2,921,219,Round numbers to three or more decimal places,214,Rounding to Decimal Places,A,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.15 \),\( 20.154 \),\( 20.253 \),-1.0,2392.0,1988.0,2330.0
3,275,306,Subtract decimals where the numbers involved h...,223,Adding and Subtracting with Decimals,A,\( 50.09-0.1= \),\( 49.99 \),\( 50.99 \),\( 50.08 \),\( 38.98 \),-1.0,699.0,2346.0,-1.0
4,416,703,Express pictorial representations of objects a...,334,Writing Ratios,C,Tom says for every one circle there are two sq...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,-1.0,1846.0,-1.0,1150.0


In [6]:
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub("@\w+", '',x)      # Delete strings starting with @
    #x = re.sub("'\d+", '',x)      # Delete Numbers
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

df_ret['input_features'] = df_ret["ConstructName"] + ". " + df_ret["SubjectName"]
df_ret['input_features'] = df_ret['input_features'].apply(lambda x: preprocess_text(x))

embedding_query = model.encode(df_ret['input_features'], convert_to_tensor=True)
misconceptions = df_misconception_mapping.MisconceptionName.values
embedding_Misconception = model.encode(misconceptions, convert_to_tensor=True)

# the first time retrieval for LLM prompt
Ret_topNids = util.semantic_search(embedding_query, embedding_Misconception, top_k=100)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [7]:
retrivals = []
dicts = {}
for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
    top_ids = Ret_topNids[idx]
    retrival = ''
    dicts[str(row['QuestionId'])] = {}
    for i, ids in enumerate(top_ids):
        # serial number + misconceptions
        retrival += f'{i+1}. ' + misconceptions[ids['corpus_id']] + '\n'
        # save retrieved misconceptions for each QuestionId.
        dicts[str(row['QuestionId'])][str(i+1)] = misconceptions[ids['corpus_id']]
    retrivals.append(retrival)

df_ret['Retrival'] = retrivals

100%|██████████| 100/100 [00:00<00:00, 1300.51it/s]


In [8]:
def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
No need to give the reasoning process and do not use "The misconception is" to start your answers.
There are some relative and possible misconceptions below to help you make the decision:

{Retrival}
"""
# just directly give your answers.

def apply_template(row, tokenizer, targetCol):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"Answer{targetCol}Text"],
                    CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"],
                    Retrival=row[f"Retrival"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

df = {}
if not IS_SUBMISSION:
    df_label = {}
    for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
        for option in ["A", "B", "C", "D"]:
            if (row.CorrectAnswer!=option) & (row[f"Misconception{option}Id"]!=-1):
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
                df_label[f"{row.QuestionId}_{option}"] = [row[f"Misconception{option}Id"]]
                
    df_label = pd.DataFrame([df_label]).T.reset_index()
    df_label.columns = ["QuestionId_Answer", "MisconceptionId"]
    df_label.to_parquet("label.parquet", index=False)
else:
    for idx, row in tqdm(df_ret.iterrows(), total=len(df_ret)):
        for option in ["A", "B", "C", "D"]:
            if row.CorrectAnswer!=option:
                df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)

df = pd.DataFrame([df]).T.reset_index()
df.columns = ["QuestionId_Answer", "text"]
df.to_parquet("submission.parquet", index=False)

100%|██████████| 100/100 [00:00<00:00, 368.89it/s]


In [9]:
print(df.loc[0, 'text'])

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Here is a question about Divide two decimals with the same number of decimal places(Multiplying and Dividing with Decimals).
Question: 0.9 \div 0.3= 
Correct Answer: 3 
Incorrect Answer: 0.3 

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
No need to give the reasoning process and do not use "The misconception is" to start your answers.
There are some relative and possible misconceptions below to help you make the decision:

1. When dividing decimals with the same number of decimal places as each other, assumes the answer also has the same number of decimal places
2. Divides instead of multiplies when multiplying by a decimal
3. Divides rather than multiplies 
4. Multiplies rather than divides
5. Divides rat

## LLM Reasoning

In [10]:
%%writefile run_vllm.py

import re
import vllm
import pandas as pd

df = pd.read_parquet("submission.parquet")

model_path = "/kaggle/input/qwen/transformers/qwen2.5-32b-instruct-awq-mmim-ref/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


responses = llm.generate(
    df["text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.8,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=512,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm=True
)

responses = [x.outputs[0].text for x in responses]
df["fullLLMText"] = responses

def extract_response(text):
    return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()

df["llmMisconception"] = responses
df.to_parquet("submission.parquet", index=False)

Writing run_vllm.py


In [11]:
!python run_vllm.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 10-29 07:52:17 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 10-29 07:52:17 config.py:715] Defaulting to use mp for distributed inference
INFO 10-29 07:52:17 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwen/transformers/qwen2.5-32b-instruct-awq-mmim-ref/1', speculative_config=None, tokenizer='/kaggle/input/qwen/transformers/qwen2.5-32b-instruct-awq-mmim-ref/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observabilit

In [12]:
llm_output = pd.read_parquet("submission.parquet")

for idx, row in llm_output[0:5].iterrows():
    print(row.llmMisconception)
    print("==="*6)

1. When dividing decimals with the same number of decimal places as each other, assumes the answer also has the same number of decimal places
1. When dividing decimals with the same number of decimal places as each other, assumes the answer also has the same number of decimal places
11. Thinks index notation represents repeated addition rather than repeated multiplication
25. Mixes up squaring and multiplying by 2 or doubling
6. Rounds to the wrong degree of accuracy (rounds too little)


In [13]:
text = llm_output.loc[0, 'text']
PREFIX = "<|im_start|>user"
text = text.split(PREFIX)[1].split("You are a Mathematics teacher.")[0].strip('\n').split('Here is a question about')[-1].strip()
print(text)

Divide two decimals with the same number of decimal places(Multiplying and Dividing with Decimals).
Question: 0.9 \div 0.3= 
Correct Answer: 3 
Incorrect Answer: 0.3


## Post-processing for LLM output

In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

df = pd.read_parquet("submission.parquet")
df_misconception_mapping = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model = SentenceTransformer('/kaggle/input/eedi-finetuned-bge-public/Eedi-finetuned-bge')

In [15]:
def number2sentence(row):
    """
    This is used for post-processing of LLM's output.
    Since we give top-N retrieval to the LLM with serial number,
    Sometimes the LLM will only output the serial number without any sentence.
    We use the 'dicts' generated at the beginning to map the serial number with corresponding misconceptions.
    """
    text = row['llmMisconception'].strip()
    # potential is the most possible serial number in LLM output.
    potential = re.search(r'^\w+\.{0,1}', text).group()
    if '.' in potential:
        sentence = text.replace(potential, '').strip()
    # if the LLM output is only a serial number, we map it with corresponding misconceptions saved in the dict.
    elif len(potential) == len(text):
        qid_retrieval = dicts[row['QuestionId']]
        try:
            # qid_retrieval is the top-N misconceptions for an QuestionId,
            # qid_retrieval[potential] is the most possible misconception.
            sentence = qid_retrieval[potential]
        except:
            # If the mapping fails, we use the first one(the most possible one in the first retrieval).
            sentence = qid_retrieval['1']
    else:
        sentence = text
        
    return sentence


df['QuestionId'] = df['QuestionId_Answer'].apply(lambda x: x.split('_')[0])
df['llmMisconception_clean'] = df.apply(number2sentence, axis=1)

In [16]:
df.head(5)

,QuestionId_Answer,text,fullLLMText,llmMisconception,QuestionId,llmMisconception_clean
0,1700_A,"<|im_start|>system\nYou are Qwen, created by A...",1. When dividing decimals with the same number...,1. When dividing decimals with the same number...,1700,When dividing decimals with the same number of...
1,1700_C,"<|im_start|>system\nYou are Qwen, created by A...",1. When dividing decimals with the same number...,1. When dividing decimals with the same number...,1700,When dividing decimals with the same number of...
2,1488_A,"<|im_start|>system\nYou are Qwen, created by A...",11. Thinks index notation represents repeated ...,11. Thinks index notation represents repeated ...,1488,Thinks index notation represents repeated addi...
3,1488_B,"<|im_start|>system\nYou are Qwen, created by A...",25. Mixes up squaring and multiplying by 2 or ...,25. Mixes up squaring and multiplying by 2 or ...,1488,Mixes up squaring and multiplying by 2 or doub...
4,921_B,"<|im_start|>system\nYou are Qwen, created by A...",6. Rounds to the wrong degree of accuracy (rou...,6. Rounds to the wrong degree of accuracy (rou...,921,Rounds to the wrong degree of accuracy (rounds...


# Second retrieval

In [17]:
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub(r"@\w+", '',x)      # Delete strings starting with @
    #x = re.sub(r"\d+", '',x)      # Delete Numbers
    x = re.sub(r"http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PREFIX = "<|im_start|>user"
df['input_features'] = df["text"].apply(lambda x: x.split(PREFIX)[1].split("You are a Mathematics teacher.")[0].strip('\n').split('Here is a question about')[-1].strip())

df['input_features'] = df['input_features'].apply(lambda x: preprocess_text(x))
df['input_features'] = df["llmMisconception_clean"] + "\n\n" + df['input_features']

embedding_query = model.encode(df['input_features'], convert_to_tensor=True)
embedding_Misconception = model.encode(df_misconception_mapping.MisconceptionName.values, convert_to_tensor=True)
top25ids = util.semantic_search(embedding_query, embedding_Misconception, top_k=25)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [18]:
df["MisconceptionId"] = [" ".join([str(x["corpus_id"]) for x in top25id]) for top25id in top25ids]

df[["QuestionId_Answer", "MisconceptionId"]].to_csv("submission.csv", index=False)
df.head()

,QuestionId_Answer,text,fullLLMText,llmMisconception,QuestionId,llmMisconception_clean,input_features,MisconceptionId
0,1700_A,"<|im_start|>system\nYou are Qwen, created by A...",1. When dividing decimals with the same number...,1. When dividing decimals with the same number...,1700,When dividing decimals with the same number of...,When dividing decimals with the same number of...,153 2273 339 151 2359 447 2542 1259 565 376 15...
1,1700_C,"<|im_start|>system\nYou are Qwen, created by A...",1. When dividing decimals with the same number...,1. When dividing decimals with the same number...,1700,When dividing decimals with the same number of...,When dividing decimals with the same number of...,153 2273 339 151 2359 447 2542 376 1259 565 15...
2,1488_A,"<|im_start|>system\nYou are Qwen, created by A...",11. Thinks index notation represents repeated ...,11. Thinks index notation represents repeated ...,1488,Thinks index notation represents repeated addi...,Thinks index notation represents repeated addi...,923 2352 952 1921 158 1884 948 2202 1227 1072 ...
3,1488_B,"<|im_start|>system\nYou are Qwen, created by A...",25. Mixes up squaring and multiplying by 2 or ...,25. Mixes up squaring and multiplying by 2 or ...,1488,Mixes up squaring and multiplying by 2 or doub...,Mixes up squaring and multiplying by 2 or doub...,2316 1137 948 902 278 1664 2175 1072 64 2352 7...
4,921_B,"<|im_start|>system\nYou are Qwen, created by A...",6. Rounds to the wrong degree of accuracy (rou...,6. Rounds to the wrong degree of accuracy (rou...,921,Rounds to the wrong degree of accuracy (rounds...,Rounds to the wrong degree of accuracy (rounds...,1315 2033 37 1248 2392 739 1529 1744 2330 1988...


## Sanity

In [19]:
if not IS_SUBMISSION:
    import pandas as pd
    from eedi_metrics import mapk
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])
    label = pd.read_parquet("label.parquet")["MisconceptionId"]
    print("Validation: ", mapk(label, predicted))

Validation:  0.459401011693326
